# R-CNN Model

### Object Detection

#### Install and Import libraries

In [1]:
%pip install tensorflow_hub
%pip install keras
import tensorflow as tf
import tensorflow_hub as hub
import matplotlib.pyplot as plt
import matplotlib.patches as patches
from six.moves.urllib.request import urlopen
from six import BytesIO
from PIL import ImageOps
from PIL import Image
import tempfile

#### Download Model

Base Model yang digunakan:
- Google Faster RCNN Inception Resnet V2
- Google Mobilenet V2

In [2]:
# Google Faster RCNN Inception Resnet V2
module_handle = "https://tfhub.dev/google/faster_rcnn/openimages_v4/inception_resnet_v2/1"

# Google Mobilenet V2
#module_handle = "https://tfhub.dev/google/openimages_v4/ssd/mobilenet_v2/1"

### Training

### Load Image

In [3]:
image_url = "https://upload.wikimedia.org/wikipedia/commons/f/fb/20130807_dublin014.JPG"

### Load Model

In [6]:
model = hub.load(module_handle)
model.signatures.keys()
detector = model.signatures['default']

### Load Functions

In [4]:
def download_and_resize_image(url, new_width=256, new_height=256):
    '''
    Fetches an image online, resizes it and saves it locally.

    Args:
        url (string) -- link to the image
        new_width (int) -- size in pixels used for resizing the width of the image
        new_height (int) -- size in pixels used for resizing the length of the image

    Returns:
        (string) -- path to the saved image
    '''


    # create a temporary file ending with ".jpg"
    _, filename = tempfile.mkstemp(suffix=".jpg")

    # opens the given URL
    response = urlopen(url)

    # reads the image fetched from the URL
    image_data = response.read()

    # puts the image data in memory buffer
    image_data = BytesIO(image_data)

    # opens the image
    pil_image = Image.open(image_data)

    # resizes the image. will crop if aspect ratio is different.
    pil_image = ImageOps.fit(pil_image, (new_width, new_height), Image.ANTIALIAS)

    # converts to the RGB colorspace
    pil_image_rgb = pil_image.convert("RGB")

    # saves the image to the temporary file created earlier
    pil_image_rgb.save(filename, format="JPEG", quality=90)

    print("Image downloaded to %s." % filename)

    return filename

downloaded_image_path = download_and_resize_image(image_url, 3872, 2592)

def load_img(path):
    '''
    Loads a JPEG image and converts it to a tensor.

    Args:
        path (string) -- path to a locally saved JPEG image

    Returns:
        (tensor) -- an image tensor
    '''

    # read the file
    img = tf.io.read_file(path)

    # convert to a tensor
    img = tf.image.decode_jpeg(img, channels=3)

    return img


def run_detector(detector, path):
    '''
    Runs inference on a local file using an object detection model.

    Args:
        detector (model) -- an object detection model loaded from TF Hub
        path (string) -- path to an image saved locally
    '''

    # load an image tensor from a local file path
    img = load_img(path)

    # add a batch dimension in front of the tensor
    converted_img  = tf.image.convert_image_dtype(img, tf.float32)[tf.newaxis, ...]

    # run inference using the model
    result = detector(converted_img)

    # save the results in a dictionary
    result = {key:value.numpy() for key,value in result.items()}

    # print results
    print("Found %d objects." % len(result["detection_scores"]))

    print(result["detection_scores"])
    print(result["detection_class_entities"])
    print(result["detection_boxes"])
    return img, result

def display_image_with_boxes(image, boxes, scores, classes, class_names, threshold=0.5):
    fig, ax = plt.subplots(1, figsize=(12, 9))
    ax.imshow(image)

    for i in range(boxes.shape[0]):
        if scores[i] >= threshold:
            y_min, x_min, y_max, x_max = boxes[i]
            h = y_max - y_min
            w = x_max - x_min
            rect = patches.Rectangle(
                (x_min * image.shape[1], y_min * image.shape[0]),
                w * image.shape[1],
                h * image.shape[0],
                linewidth=2,
                edgecolor='r',
                facecolor='none'
            )
            ax.add_patch(rect)
            plt.text(
                x_min * image.shape[1],
                y_min * image.shape[0] - 5,
                f'{class_names[i].decode("utf-8")}: {scores[i]:.2f}',
                color='red',
                fontsize=12,
                bbox=dict(facecolor='yellow', alpha=0.5)
            )
    plt.show()

Image downloaded to /tmp/tmp_ypah9xi.jpg.


<ipython-input-4-950e965d73e2>:31: DeprecationWarning: ANTIALIAS is deprecated and will be removed in Pillow 10 (2023-07-01). Use LANCZOS or Resampling.LANCZOS instead.
  pil_image = ImageOps.fit(pil_image, (new_width, new_height), Image.ANTIALIAS)


### Run Inference

In [5]:
# runs the object detection model and prints information about the objects found
img, result = run_detector(detector, downloaded_image_path)
display_image_with_boxes(
    img.numpy(),
    result['detection_boxes'],
    result['detection_scores'],
    result['detection_class_entities'],
    result['detection_class_entities']
    )

NameError: name 'detector' is not defined